**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4034 - Análisis de grandes volúmenes de datos (Gpo 10)**

Tecnológico de Monterrey

Dr. Nestor Velasco Bermeo

**Semana 9**
9.5 Avance de proyecto 4: Sistema de Recomendación

---


*   NOMBRE: Luis Salomon Flores Ugalde
*   MATRÍCULA: A00817435
*   NOMBRE: Alejandro Guzmán Chávez
*   MATRÍCULA: A01795398
*   NOMBRE: Oscar Israel Lerma Franco
*   MATRÍCULA: A01380817

El desarrollo de esta actividad contribuye al cumplimiento de los objetivos del Modulo 5:

Aplicar algoritmos de machine learning a big data enfocado al modelado predictivo y toma de decisiones basada en datos.
Identificar la intersección entre Big Data e Inteligencia Artificial.
Reconocer la aplicación de algoritmos de machine learning en el análisis de Big Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import json
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/Big data/Big Data Equipo"
os.chdir(DIR)
print("Current working directory:", os.getcwd())


Mounted at /content/drive
Current working directory: /content/drive/.shortcut-targets-by-id/1P01N8nYF_gZG8BKKyp05LKIheTMUxEJ0/Big Data Equipo


In [3]:
# Leer los datos con formato correcto
steam_games_df_base = pd.read_json('steam_games_df.json', lines=True)
reviews_df_base = pd.read_json('reviews_df.json', lines=True)
user_items_df_base = pd.read_json('user_items_df.json', lines=True)
# Copiar los dataframes
steam_games_df = steam_games_df_base
reviews_df = reviews_df_base
user_items_df = user_items_df_base
# Revisamos uno de los dataframes
steam_games_df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,item_id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140.0,Kotoshiro,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290.0,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400.0,彼岸领域,None
4,,[],Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570.0,None,None


## Preprocesamiento

### Steam games

In [4]:
#steam games
steam_games_df = steam_games_df[steam_games_df['item_id'].notna()]
steam_games_df.reset_index(drop=True, inplace=True)
steam_games_df['item_id'] = steam_games_df['item_id'].astype(int)

<ipython-input-4-1239f64f6ad7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games_df['item_id'] = steam_games_df['item_id'].astype(int)


In [5]:
steam_games_df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,item_id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,None
4,,[],Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,None


In [6]:
# Contamos los generos y los tags, ya que son las características que ayudarán a encontrar similitudes
from collections import Counter

contador_generos = Counter(g for genres in steam_games_df['genres'] for g in genres)
contador_tags = Counter(t for tags in steam_games_df['tags'] for t in tags)

### Reviews

In [7]:
#reviews
reviews_df = reviews_df[reviews_df['item_id'].notna()]
reviews_df.reset_index(drop=True, inplace=True)
reviews_df['item_id'] = reviews_df['item_id'].astype(int)
#Si no esta en nuestra base de steam games.
reviews_df = reviews_df[reviews_df['item_id'].isin(steam_games_df['item_id'])]
reviews_df = reviews_df.drop(columns=['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1)

<ipython-input-7-f9651aa9f02b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['item_id'] = reviews_df['item_id'].astype(int)


In [8]:
# Vemos cómo quedó reviews
reviews_df.head()

,item_id,recommend,review,user_id
0,1250,1.0,Simple yet with great replayability. In my opi...,76561197970982479
1,22200,1.0,It's unique and worth a playthrough.,76561197970982479
4,227300,1.0,For a simple (it's actually not all that simpl...,js41637
5,239030,1.0,Very fun little game to play when your bored o...,js41637
6,248820,1.0,A suitably punishing roguelike platformer. Wi...,evcentric


### User items

In [9]:
#User Items
user_items_df = user_items_df[user_items_df['item_id'].isin(steam_games_df['item_id'])]
user_items_df['item_id'] = user_items_df['item_id'].astype(int)
user_items_df = user_items_df.drop(columns=['user_url'], axis=1)

<ipython-input-9-9aaa93349e1d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items_df['item_id'] = user_items_df['item_id'].astype(int)


In [10]:
def sigmoid(x, scale=1):
    return 1 / (1 + np.exp(-x * scale))
user_items_df['log_ptime'] = np.log1p(user_items_df['playtime_forever'])
user_items_df['exp_ptime_2week'] = sigmoid(user_items_df['playtime_2weeks'], scale=0.3)
max_value = 10
user_items_df['exp_ptime_2week'] *= max_value

In [11]:
scaler_playtime = MinMaxScaler(feature_range=(0, 5))
user_items_df['log_ptime_scaled'] = scaler_playtime.fit_transform(user_items_df[['log_ptime']].values.reshape(-1, 1))

scaler_lastweek = MinMaxScaler(feature_range=(0, 5))
user_items_df['exp_ptime_2week_scaled'] = scaler_lastweek.fit_transform(user_items_df[['exp_ptime_2week']].values.reshape(-1, 1))

In [12]:
n=0.6 # Peso para el tiempo de juego en las últimas dos semanas
m=0.4 # Peso para el tiempo de juego total
user_items_df['combined_rating'] = n * user_items_df['exp_ptime_2week_scaled'] + m * user_items_df['log_ptime_scaled']

In [13]:
user_items_df = pd.merge(user_items_df, reviews_df[['user_id', 'item_id', 'recommend']], on=['user_id', 'item_id'], how='left')
user_items_df['recommend'].fillna(0, inplace=True)

In [14]:
user_items_df['weighted_rating'] = user_items_df['combined_rating'] * (1 + user_items_df['recommend'])


In [15]:
user_items_df.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,log_ptime,exp_ptime_2week,log_ptime_scaled,exp_ptime_2week_scaled,combined_rating,recommend,weighted_rating
0,10,Counter-Strike,6.0,0.0,76561197970982479,1.945910,5.0,0.727522,0.0,0.291009,0.0,0.291009
1,20,Team Fortress Classic,0.0,0.0,76561197970982479,0.000000,5.0,0.000000,0.0,0.000000,0.0,0.000000
2,30,Day of Defeat,7.0,0.0,76561197970982479,2.079442,5.0,0.777446,0.0,0.310978,0.0,0.310978
3,40,Deathmatch Classic,0.0,0.0,76561197970982479,0.000000,5.0,0.000000,0.0,0.000000,0.0,0.000000
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,0.000000,5.0,0.000000,0.0,0.000000,0.0,0.000000


## Algoritmo de filtrado basado en contenido: Similaridad de Coseno

Para construir un sistema de recomendacion con filtrado basado en contenido necesitamos preparar nuestro dataset de forma tal que las filas representen los juegos de steam y las columnas las caracteristicas (generos y tags)


Para construir este sistema de recomendacion item-item utilizaremos una metrica llamada similitud coseno `cosine similarity`.

La similitud Coseno se basa en el coseno del angulo entre dos vectores (c.e., $A$ y $B$). Entre mas pequeño es el angulo, mayor es la similitud entre los vectores, y se calcula de la siguiente forma:

$$\cos(\theta) = \frac{A\cdot B}{||A|| ||B||}$$

En este ejercicio, utilizaremos la funcion de scikit-learn de `cosine_similarity` para generar la matriz de similitud de tipo $(n_{\text{juego}}, n_{\text{juego}})$. Esta matriz nos permitira extraer juegos que sean parecidos al videojuego de interes.

In [16]:
genres = list(contador_generos.keys())

for g in genres:
    steam_games_df[g] = steam_games_df['genres'].transform(lambda x: int(g in x))

<ipython-input-16-c41eb6c072e1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games_df[g] = steam_games_df['genres'].transform(lambda x: int(g in x))
<ipython-input-16-c41eb6c072e1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games_df[g] = steam_games_df['genres'].transform(lambda x: int(g in x))
<ipython-input-16-c41eb6c072e1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [17]:
steam_games_df[genres].head()

,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,[,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Para codificar los tags usaremos otro algoritmo para mejorar el rendimiento ya que son muchos tags

In [18]:
tags = list(contador_tags.keys())

tags_matrix = {tag: steam_games_df['tags'].apply(lambda x: int(tag in x)) for tag in contador_tags.keys()}
tags_df = pd.DataFrame(tags_matrix)
steam_features = pd.concat([steam_games_df[genres], tags_df], axis=1)

steam_features.head()

,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,[,...,Foreign,Feature Film,Hardware,Steam Machine,Philisophical,Mystery Dungeon,Wrestling,360 Video,Faith,Bikes
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
start_time_cosine_sim = time.time()
cosine_sim = cosine_similarity(steam_features, steam_features)
end_time_cosine_sim = time.time()
print(f"Las dimensiones de similaridad coseno de las caracteristicas de nuestra matriz de similitud son: {cosine_sim.shape}")

Las dimensiones de similaridad coseno de las caracteristicas de nuestra matriz de similitud son: (32133, 32133)


Ahora generamos un método que devuelva las recomendaciones para un usuario específico

In [20]:
game_idx = dict(zip(steam_games_df['item_id'], list(steam_games_df.index)))


In [21]:
def encuentra_juego(game_id):
    title = steam_games_df[steam_games_df['item_id'] == game_id]['title'].iloc[0]
    return title

In [22]:
def obtener_recomendaciones_basadas_contenido(game_id, numero_recomendaciones=10):
    title = encuentra_juego(game_id)
    idx = game_idx[game_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(numero_recomendaciones+1)]
    similar_games = [i[0] for i in sim_scores]
    print(f"Porque te gustó {title} te pueden gustar:")
    print(steam_games_df['title'].iloc[similar_games])

### Recomendaciones Cosine

In [23]:
for index,row in reviews_df.head().iterrows():
  print(f'\nRecomendaciones para el usuario: {row["user_id"]}')
  obtener_recomendaciones_basadas_contenido(row['item_id'],5)


Recomendaciones para el usuario: 76561197970982479
Porque te gustó Killing Floor te pueden gustar:
7987                                       Killing Floor 2
31527                                        Left 4 Dead 2
31753                           Call of Duty: World at War
31752                                          Left 4 Dead
1380     Resident Evil Revelations / Biohazard Revelations
Name: title, dtype: object

Recomendaciones para el usuario: 76561197970982479
Porque te gustó Zeno Clash te pueden gustar:
30194    Zeno Clash 2
25058      Orc Slayer
26728         Despair
6581          WARZONE
1850       CONSORTIUM
Name: title, dtype: object

Recomendaciones para el usuario: js41637
Porque te gustó Euro Truck Simulator 2 te pueden gustar:
30303                    Euro Truck Simulator
24194                American Truck Simulator
26714    Euro Truck Simulator 2 - Scandinavia
14744                               Real Farm
5844                                   Jalopy
Name: title, dt

## Algoritmo de recomendación filtrado colaborativo: SVD basico

In [24]:
reviews_svd_df = reviews_df[['user_id', 'review', 'recommend','item_id']]

In [25]:
start_time_svd_basic = time.time()
UtAus_review = reviews_svd_df.pivot_table(values='recommend', index='item_id', columns='user_id', fill_value=0).astype(float)
print('(juego, usuarios) =', (UtAus_review.shape))
UtAus_review.head()

(juego, usuarios) = (3195, 24380)


user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-I_AM_EPIC-,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zunbae,zuzuga2003,zv_odd,zvanik,zwanzigdrei,zy0705,zynxgameth,zyr0n1c
item_id,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
sparse_matrix_svd = csr_matrix(UtAus_review.values)

U, sigma, VT = svds(sparse_matrix_svd, k=450)

sigma = np.diag(sigma)

# Compute the dot product to get the approximated original matrix
predicted_ratings = np.dot(np.dot(U, sigma), VT)

# Converting the reconstructed matrix back to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=UtAus_review.index, columns=UtAus_review.columns)

In [27]:
actual_ratings = UtAus_review.values.flatten()
predicted_ratings = predicted_ratings_df.values.flatten()

# Only consider non-zero elements
mask = actual_ratings > 0
actual_ratings = actual_ratings[mask]
predicted_ratings = predicted_ratings[mask]
end_time_svd_basic = time.time()

# Calculate RMSE
rmse = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("RMSE:", rmse)

RMSE: 0.3505428989509815


In [28]:
def recommend_items_svd(user_id, original_matrix, predictions_df, num_recommendations=5):
    user_col_number = original_matrix.columns.get_loc(user_id)
    sorted_user_predictions = predictions_df.iloc[:, user_col_number].sort_values(ascending=False)
    #print(sorted_user_predictions)
    user_data = original_matrix[user_id]
    #print(user_data[user_data != 0].index)
    return sorted_user_predictions.drop(user_data[user_data != 0].index).head(num_recommendations)
    # return recommendations.head(num_recommendations)

### Recomendaciones SVD

In [30]:
user_id = 'LydiaMorley'
recommendations = recommend_items_svd(user_id, UtAus_review, predicted_ratings_df)

recommendations_df = pd.DataFrame(recommendations).reset_index()
recommendations_df.columns = ['item_id', 'predicted_rating']
merged_df = recommendations_df.merge(steam_games_df, left_on='item_id', right_on='item_id', how='left')
merged_df

,item_id,predicted_rating,publisher,genres,app_name,title,url,release_date,tags,discount_price,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,200170,0.044101,Team17 Digital Ltd,[Strategy],Worms Revolution,Worms Revolution,http://store.steampowered.com/app/200170/Worms...,2012-10-10,"[Strategy, Multiplayer, Funny, Turn-Based Stra...",NaN,...,0,0,0,0,0,0,0,0,0,0
1,17510,0.043667,,[Indie],Age of Chivalry,Age of Chivalry,http://store.steampowered.com/app/17510/Age_of...,2007-12-01,"[Indie, Free to Play, Medieval, Mod, Action, M...",NaN,...,0,0,0,0,0,0,0,0,0,0
2,413410,0.041808,"Spike Chunsoft Co., Ltd.",[Adventure],Danganronpa: Trigger Happy Havoc,Danganronpa: Trigger Happy Havoc,http://store.steampowered.com/app/413410/Danga...,2016-02-18,"[Anime, Mystery, Visual Novel, Detective, Stor...",NaN,...,0,0,0,0,0,0,0,0,0,0
3,349480,0.038164,Steam Greenlight,"[Action, Free to Play, RPG]",Modular Combat,Modular Combat,http://store.steampowered.com/app/349480/Modul...,2008-01-18,"[Free to Play, Multiplayer, Action, Mod, FPS, ...",NaN,...,0,0,0,0,0,0,0,0,0,0
4,102700,0.036182,En Masse Entertainment,"[Action, Free to Play]",A.V.A. Alliance of Valiant Arms™,A.V.A. Alliance of Valiant Arms™,http://store.steampowered.com/app/102700/AVA_A...,2011-06-14,"[Free to Play, FPS, Action, Multiplayer, Shoot...",NaN,...,0,0,0,0,0,0,0,0,0,0


## Metricas

In [34]:
def get_recom_content_eval(game_id, recom=5):
    #title = encuentra_juego(game_id)
    idx = game_idx[game_id]
    sim_scores = cosine_sim[idx]
    recom_eval = pd.DataFrame({'index': steam_games_df.index,'similarity': sim_scores})
    #similarity_df = pd.DataFrame(list(enumerate(cosine_sim[idx])), columns=['index', 'similarity'])

    recom_eval = recom_eval.nlargest(recom,'similarity')
    recom_eval['game_id'] = recom_eval['index'].map(steam_games_df['item_id'])

    return recom_eval


def get_recom_content_user(game_id, numero_recom=5):
    full_recom_df = get_recom_content_eval(game_id, numero_recom + 1)
    if game_id in full_recom_df['game_id'].values:
        full_recom_df = full_recom_df[full_recom_df['game_id'] != game_id]

    return full_recom_df.head(numero_recom)


In [35]:
def gen_recom_ground_truth(numero_recom):
    recommendations = {}
    ground_truth = {}

    for user_id in reviews_df['user_id'].unique():
        user_reviews = reviews_df[reviews_df['user_id'] == user_id]
        ground_truth[user_id] = user_reviews[user_reviews['recommend'] == True]['item_id'].tolist()

        user_recommendations = set()
        for game_id in user_reviews['item_id']:
            recommendations_for_game = get_recom_content_eval(game_id, numero_recom)
            if not recommendations_for_game.empty:
                user_recommendations.update(recommendations_for_game['game_id'].tolist())

        recommendations[user_id] = list(user_recommendations)[:numero_recom]

    return recommendations, ground_truth

def gen_recom_ground_truth_svd(numero_recom):
    recommendations = {}
    ground_truth = {}

    for user_id in reviews_df['user_id'].unique():
        user_reviews = reviews_df[reviews_df['user_id'] == user_id]
        ground_truth[user_id] = user_reviews[user_reviews['recommend'] == True]['item_id'].tolist()

        # Obtener recomendaciones para el usuario usando SVD
        recommended_items = recommend_items_svd(user_id, UtAus_review, predicted_ratings_df, num_recommendations=numero_recom)

        # Convertir las puntuaciones predichas a solo IDs de juegos
        user_recommendations = list(recommended_items.index)

        recommendations[user_id] = user_recommendations[:numero_recom]

    return recommendations, ground_truth


In [36]:
start_time_rec_cosine = time.time()
recommendations_dict, ground_truth_dict = gen_recom_ground_truth( numero_recom=5)
end_time_rec_cosine = time.time()

start_time_rec_svd = time.time()
recommendations_dict_svd, ground_truth_dict_svd = gen_recom_ground_truth_svd(numero_recom=5)
end_time_rec_svd = time.time()

#Cosine Similarity
recommendations = [recommendations_dict[user] for user in recommendations_dict]
ground_truth = [ground_truth_dict[user] for user in ground_truth_dict]

#SVD simple
recommendations_svd = [recommendations_dict_svd[user] for user in recommendations_dict_svd]
ground_truth_svd = [ground_truth_dict_svd[user] for user in ground_truth_dict_svd]


In [37]:
#import recmetrics

def precision_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / k

def recall_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / len(actual)

def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def ndcg_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    idcg = sum([1.0 / np.log2(i + 2) for i in range(len(actual))])
    dcg = sum([1.0 / np.log2(i + 2) for i, p in enumerate(predicted) if p in actual])
    return dcg / idcg

def mrr_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0

    predicted = predicted[:k]
    for i, p in enumerate(predicted):
        if p in actual:
            return 1.0 / (i + 1.0)
    return 0.0

def calculate_rmse(sparse_matrix, k):
    U, sigma, VT = svds(sparse_matrix, k=k)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), VT)

    actual_ratings = sparse_matrix.toarray().flatten()
    predicted_ratings = predicted_ratings.flatten()

    mask = actual_ratings > 0
    actual_ratings = actual_ratings[mask]
    predicted_ratings = predicted_ratings[mask]

    rmse = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
    return rmse

def evaluate_recommendations(ground_truth, recommendations, k=5):
    metrics = {
        'Precision@K': np.mean([precision_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'Recall@K': np.mean([recall_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'MAP@K': mapk(ground_truth, recommendations, k),
        'NDCG@K': np.mean([ndcg_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'MRR@K': np.mean([mrr_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)])
    }
    return metrics

metrics_cosine = evaluate_recommendations(ground_truth, recommendations, k=5)
metrics_svd = evaluate_recommendations(ground_truth_svd, recommendations_svd, k=5)

In [38]:
duration_training_svd = end_time_svd_basic - start_time_svd_basic
duration_training_cosine = end_time_cosine_sim - start_time_cosine_sim
duration_recommendation_svd = end_time_rec_svd - start_time_rec_svd
duration_recommendation_cosine = end_time_rec_cosine - start_time_rec_cosine

metrics_df_simple = pd.DataFrame({
    'Metric': ['Precision@K', 'Recall@K', 'MAP@K', 'NDCG@K', 'MRR@K',
               'Tiempo Entrenamiento', 'Tiempo Recomendación'],
    'Cosine Method': [
        metrics_cosine['Precision@K'],
        metrics_cosine['Recall@K'],
        metrics_cosine['MAP@K'],
        metrics_cosine['NDCG@K'],
        metrics_cosine['MRR@K'],
        duration_training_cosine,
        duration_recommendation_cosine
    ],
    'SVD Method': [
        metrics_svd['Precision@K'],
        metrics_svd['Recall@K'],
        metrics_svd['MAP@K'],
        metrics_svd['NDCG@K'],
        metrics_svd['MRR@K'],
        duration_training_svd,
        duration_recommendation_svd
    ]
})

metrics_df_simple

,Metric,Cosine Method,SVD Method
0,Precision@K,0.176743,0.000000
1,Recall@K,0.647006,0.000000
2,MAP@K,0.307413,0.000000
3,NDCG@K,0.410834,0.000000
4,MRR@K,0.388318,0.000000
5,Tiempo Entrenamiento,3.558053,202.208314
6,Tiempo Recomendación,269.419374,141.044295


In [41]:
merged_df = reviews_df.merge(steam_games_df, left_on='item_id', right_on='item_id', how='left')

# Filter to get liked games by each user
liked_games_df = merged_df[merged_df['recommend'] == True]

# Prepare the output
recommendations_output = {}
for user_id, rec_ids in recommendations_dict_svd.items():
    # Get the names of games the user likes
    user_liked_games = liked_games_df[liked_games_df['user_id'] == user_id]['title'].tolist()

    # Get the names of the recommended games
    recommended_games_names = steam_games_df[steam_games_df['item_id'].isin(rec_ids)]['title'].tolist()

    # Store in dict
    recommendations_output[user_id] = {
        'Liked Games': user_liked_games,
        'Recommended Games': recommended_games_names
    }


In [43]:

for user_id, games_info in recommendations_output.items():
    if user_id in ['js41637', 'evcentric']:  # Limit output to specific users for example
        print(f"{user_id} Le gustaron estos juegos: {', '.join(games_info['Liked Games'])}")
        print(f"Puede que le interesaran estos: {', '.join(games_info['Recommended Games'])}\n")

js41637 Le gustaron estos juegos: Euro Truck Simulator 2, Papers, Please
Puede que le interesaran estos: MINERVA, Enforcer: Police Crime Action, PAYDAY 2: Hotline Miami, Ford Racing 3, S.T.A.L.K.E.R.: Call of Pripyat

evcentric Le gustaron estos juegos: Risk of Rain, TIS-100, Transistor, 3089 -- Futuristic Action RPG, Space Pirates and Zombies, Gnomoria
Puede que le interesaran estos: Command &amp; Conquer: Red Alert 3, Serena, Endless Legend™, Dr. Langeskov, The Tiger, and The Terribly Cursed Emerald: A Whirlwind Heist, Party Hard



## Bibliografía

1. Marabelli, M; Saunders, C; y Wiener, M. (2020). Big-data business models: A critical literature review and multiperspective research framework. 35(1), 66-91.  https://journals.sagepub.com/doi/reader/10.1177/0268396219896811.
2. Kang W., McAuley J. (ICDM, 2018) Self-attentive sequential recommendation. UC San Diego pdf